# Masterthesis
## Reporting - explained Varianz and create plot for relationship

This script create plots to show the relationship between dataset in a scenariofile.

**Imports and Definitions**
- The necessary libraries are loaded here and important variables are defined.

**Imports and settings for this script**
- Import libraries and set variables for this script.

**Function for calculate the variance**
- This function convert the dataframe to mp format

**Calculate varianz**
- In this step, each scenario file is read in and the variance is calculated

**Function to convert pandas dataframe to Mp format**
- This function convert the dataframe to mp format

**Create 3D-Plot**
- In this script read the datafile, convert the data mp-format and create the plot.

**Create plots**
- Read the scenario- and mapping-file to create for each scenario a plot. This plot shows the relationship between the datasetfile in a scenario.

- For this purpose, the arithmetic mean value is calculated for the subcarrier of a data set and displayed graphically.

## Imports and Definitions

In [ ]:
# Import sklearn
import sklearn

# Import pandas
import pandas as pd

# Import numpy
import numpy as np

# To calculate amplitude and phase
import math

# Measure runtime of a jupyter jotebook code cell
from timeit import default_timer as timer

# Used to check if file exists
import os

# Used to check if directory exists
import pathlib

# Import Operation System Calls
import SubOperationSystem

# check os
if os.name == 'nt':
    print("OS is Windows")
    Delimiter = '\\'
    
else:
    print("OS is Linux")
    Delimiter = '/'
    
# Path of datasets (root directory)
PathDataset = 'Dataset' + Delimiter    

# Path of datasets
PathDatasetSub = PathDataset + 'CsiFilesRah' + Delimiter
        
# Path of the converted files
PathConverted = PathDataset + 'Converted' + Delimiter

# Set path for scenario files
PathScenario = PathDataset + 'Scenario' + Delimiter

# Set path for scenario files
PathResult = PathDataset + 'Result' + Delimiter

# Set path for scenario files
PathPlot = PathDataset + 'Plot' + Delimiter

# Set path for scenario files
PathConfig = 'FilesConfig' + Delimiter

# Scenariofile (file with info about the ten scenarios)
FileScenario = 'FileScenario.csv'

# Mappingfile (file with info about original and converted filenames)
FileMapping = 'FileMapping.csv'

# Imports and settings for this script

In [ ]:
# Import StandardScaler
from sklearn.preprocessing import StandardScaler

# Import PCA
from sklearn.decomposition import PCA

# Result Files
FileResultFilterHampel = "FileResultFilterHampel.csv"

# Import matplotlib
import matplotlib.pyplot as plt

# Extension for this plots
FileExtension = '_relation'

# Function for variance

In [ ]:
# Def function Varianzce
# X are the samples without label
# n is the count of components
def Variance(X, n):
    
    # Create instance for scaler
    scaler = StandardScaler()
    
    # Create instance for pca
    pca = PCA(n_components = n)
    
    # Fit and fit_transform X
    pca.fit(scaler.fit_transform(X))

    # return of explained variance (Sum)
    return pca.explained_variance_ratio_.cumsum()[-1:]

# Calculate variance

In [ ]:
# open scenario mapping file to read
dfScenario = pd.read_csv(PathConfig + FileScenario, index_col=0)

# loop through scenario dataframe
for ind in dfScenario.index:
    
    # get scenarios and dataset from dataframe
    Scenario = (dfScenario['Scenario'][ind])
    
    # read scenario files
    df = pd.read_csv(PathScenario + Scenario + "_ah.csv")

    # drop label column
    X = df.drop(columns=['label']).values

    # Print scenario name
    print(Scenario)
    print(80 * '=')
        
    # Loop through count of features
    for f in range(1,53):
        
        # Print result
        print('Components:\t', f, '=\t', Variance(X, f), '\tCumulative Variance')
    
    # Print line
    print()

## Function to convert pandas dataframe to Mp format

In [ ]:
# Convert the pandas dataframe to mathplot format
def convertPd2Mp(df: pd.DataFrame):

    # Get index of dataframe
    index = df.index
    
    # Get columnnames
    columns = df.columns

    # 
    x, y = np.meshgrid(np.arange(len(columns)), np.arange(len(index)))
    
    #
    z = np.array([[df[c][i] for c in columns] for i in index])
      
    # Return values
    return x, y, z

## Create 3D-Plot

In [ ]:
# Set Datafile

# File withoutliers
# datafile = 'Person-1_Gehen_langsam_LLOS_ah.csv'

# File with outliers
datafile = 'Person-1_Gehen_langsam_LLOS_a.csv'

# Load data file
df = pd.read_csv(PathConverted + Delimiter + datafile)

# Remove column "label"
df = df.drop("label",axis=1)

# Convert dataframe df to mathplot format
x, y, z = convertPd2Mp(df)

# Set figure size
fig = plt.figure(figsize=(10,10))

# Plot in 3D
axes = fig.add_subplot(projection='3d')

# Plot axis
axes.plot_surface(x, y, z, cmap='viridis', edgecolor='none')

# Set name for labels (x,y,z)
axes.set_xlabel('Columns')
axes.set_ylabel('Raws', rotation = 45)
axes.set_zlabel('Amplitude')

# Set zero line
axes.set_zlim3d(bottom=0)

# save scenario
plt.savefig(PathPlot + "Plot3D.png", bbox_inches = "tight")

# Show plot
plt.show()

# Print max value
ArrayCsiFiltered = df.to_numpy()
print("Max Value of Amplitude is:", np.max(ArrayCsiFiltered))

# Create relation plots

In [ ]:
# Read the scenario mapping file
dfScenario = pd.read_csv(PathConfig + FileScenario, index_col=0)

# Read the mapping file
dfFiles = pd.read_csv(PathConfig + FileMapping, index_col=0)

# loop through scenarios
for ind in dfScenario.index:
    
    # Get scenarios and dataset from scenario file
    Scenario,Datasets = (dfScenario['Scenario'][ind], dfScenario['Datasets'][ind])

    # Split to dataset items because we need a int value
    DatasetItems=list(Datasets.split())
        
    # Create enty list
    ListValues = []
    ListFilenames = []
        
    # loop through dataitems
    for DatasetItem in DatasetItems:
        
        # Filenames of needed dataset in the column 'FilenameNew'
        DatasetFilenames = dfFiles.loc[int(DatasetItem)]['FilenameNew']
        
        # Remove special characters
        DatasetFilenamesLabel = DatasetFilenames.replace("_", " ")
        DatasetFilenamesLabel = DatasetFilenamesLabel.replace("-", " ")
        DatasetFilenamesLabel = DatasetFilenamesLabel.replace(".csv", "")

        # Get filename
        ListFilenames.append(DatasetFilenamesLabel)
        
        # Rename filename with amplitude label
        FileDataHampel = DatasetFilenames.replace(".csv", "_ah.csv")
        
        # load dataset
        df = pd.read_csv(PathConverted + FileDataHampel)

        # Create list (features)
        ListFeatures = list(range(1,53))
        
        # label for y (as copy)
        y = df['label'].copy()
        
        # Create a new list for mean values
        ListMeanValue = []
        
        # Get mean value from dataframe
        ListMeanValue = df.mean()
        
        # Remove first row
        ListMeanValue = ListMeanValue[:-1]
    
        # append list
        ListValues.append(ListMeanValue)
    
    # for all entrys in list
    for i, ax in enumerate(ListValues):
        
        # plot each curve
        plt.plot(ListFeatures, ax)
    
    # add legend
    plt.legend(ListFilenames, bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Cleanup title
    TitleScenario = Scenario.replace("Scenario", "Szenario ")
    TitleScenario = TitleScenario.replace("10", "xx")
    TitleScenario = TitleScenario.replace("0", " ")
    TitleScenario = TitleScenario.replace("xx", "10")
    
    # add title
    plt.suptitle(TitleScenario)
    
    # set label
    plt.xlabel('Features')
    plt.ylabel('Arithmetischer Mittelwert der Amplitude')
         
    # save scenario
    plt.savefig(PathPlot + Scenario + FileExtension + ".png", bbox_inches = "tight")
    
    # show scenario
    plt.show()


## Create a plot to show the differences between the data sets

In [ ]:
# Set File names
File01 = 'Person-5_Gehen_langsam_LLOS_ah.csv'
File02 = 'Person-5_Gehen_normal_LLOS_ah.csv'
File03 = 'Person-5_Gehen_schnell_LLOS_ah.csv'

# load dataset
dfFile01 = pd.read_csv(PathConverted + File01)
dfFile02 = pd.read_csv(PathConverted + File02)
dfFile03 = pd.read_csv(PathConverted + File03)

# Create dataset with the differences between the dataset
df1 = dfFile01.subtract(dfFile02)
df2 = dfFile01.subtract(dfFile03)
df3 = dfFile03.subtract(dfFile02)

# Create a list (features)
ListFeatures = list(range(1,53))

# label for y
y = df['label']

# Create a new list for mean values
ListMeanValue1 = []
ListMeanValue2 = []
ListMeanValue3 = []

# Create enty list
ListValues1 = []
ListValues2 = []
ListValues3 = []

# Get mean value from dataframe
ListMeanValue1 = df1.mean()
ListMeanValue2 = df2.mean()
ListMeanValue3 = df3.mean()

# Remove first row from the datasets
ListMeanValue1 = ListMeanValue1[:-1]
ListMeanValue2 = ListMeanValue2[:-1]
ListMeanValue3 = ListMeanValue3[:-1]

# append values to list
ListValues1.append(ListMeanValue1)
ListValues2.append(ListMeanValue2)
ListValues3.append(ListMeanValue3)

# for all entrys in list
for i, ax in enumerate(ListValues1):
    # plot each curve
    plt.plot(ListFeatures, ax)

# for all entrys in list
for i, bx in enumerate(ListValues2):
    # plot each curve
    plt.plot(ListFeatures, bx)

# for all entrys in list
for i, cx in enumerate(ListValues3):
    # plot each curve
    plt.plot(ListFeatures, cx)
    
# Add title to plot
plt.suptitle('Szenario 4')

# set label
plt.xlabel('Features')
plt.ylabel('Arithmetischer Mittelwert der Amplitude')

# Set legend
File01 = 'Unterschied zwischen\nGehen langsam und normal'
File02 = 'Unterschied zwischen\nGehen langsam und schnell'
File03 = 'Unterschied zwischen\nGehen schnell und normal'

# Set legend
plt.gca().legend((File01,File02,File03), bbox_to_anchor=(1.3, 1), loc='upper center')

# Set zero line
plt.axhline(y=0,color='black',linewidth = 0.5)

# Save result to file
plt.savefig(PathPlot + "DiffencesBetweenDatasets.png", bbox_inches = "tight")

# show scenario
plt.show()
